In [9]:
'''
* Import modules
'''
import pandas as pd
pd.options.display.max_rows = 4000 # To display all elements


In [10]:
'''
* Read the raw data.
'''    
import pandas as pd

# Read data
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
# Easy to handle a data frame.
df = pd.DataFrame( df, columns=['text'])

### Info
df.info()
df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 1 columns):
text    500 non-null object
dtypes: object(1)
memory usage: 4.0+ KB


text    03/25/93 Total time of visit (in minutes):\n
Name: 0, dtype: object

In [11]:
'''
* Clean the dataset
'''

# Remove decimal numbers to avoid possible conflict with date/year/month
df['text'] = df['text'].str.replace(r'\d{1,4}\.\d{1,4}', '???')

In [12]:
'''
* Define building blocks for the date, month and year patterns.
'''

# Spacing
single_spacing = r'[ \-,.():;/\n]'
double_spacing = r'[ \-,.():;/\n]?[ \-,.():;/\n]'

# Suffixes
suffixes = r'[(st)(rd)(th)]'

# Months
months = r'(?P<month>[1-9]|0[1-9]|1[012])' # It fixes all.

# Years
years = r'(?P<year>\d{4}|\d{2})'

# Years_only-- 4 length
years_only = r'(?P<year>\d{4})'

# Dates
dates = r'(?P<date>[1-9]|0[1-9]|[12]\d|3[01])' # Fixes all.

#Months names
months_name = r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)'

# Dict to convert names to number
month_dict = {'Jan':1, 'January':1, 'Janaury' :1, #Note multiple spellings to cover human mistakes
              'Feb':2, 'February':2,
              'Mar':3, 'March':3, 
              'Apr':4, 'April':4, 
              'May':5,
              'Jun':6, 'June':6,
              'Jul':7, 'July':7,
              'Aug':8, 'August':8, 
              'Sep':9, 'September':9,
              'Oct':10, 'October':10,
              'Nov':11, 'November':11, 
              'Dec':12, 'December':12, 'Decemeber':12
             }

In [13]:
'''
* Combine pattern to search possible date patterns:
04/20/2009; 04/20/09; 4/20/09; 4/3/09
Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 
Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
Feb 2009; Sep 2009; Oct 2010
6/2008; 12/2009
2009; 2010
'''

pat0 = months+single_spacing+dates+single_spacing+years
pat1 = months_name+single_spacing+dates+double_spacing+years_only
pat2 = dates+single_spacing+months_name+single_spacing+years_only
pat3 = months_name+double_spacing+years_only
pat4 = months+double_spacing+years_only
pat5 = years_only+double_spacing

######### Search the patterns and combine
pat = [pat0, pat1, pat2, pat3, pat4, pat5]

for i,ipat in enumerate(pat):
    # Search the pattern in the Pandas series.
    out = df['text'].str.extractall( ipat )
    # Remove the entries that have been used.
    bool_index = ~df.index.isin([index[0] for index in out.index])
    df = df[bool_index]
    # Replace months name by numbers
    out.replace(month_dict, inplace=True)
    for icol in out.columns:
        out[icol] = out[icol].astype(int)
    # Combine
    if i == 0:
        target_df = out
    else:
        target_df = pd.concat([target_df, out], ignore_index=False, sort=False ) # It work for diffrent order. Notes it changes datatype to fload because NAN is float.

target_df['date'] = target_df['date'].fillna(1).astype(int)
target_df['month'] = target_df['month'].fillna(1).astype(int)
target_df['year'] = target_df['year'].apply(lambda x: 1900 + x if x <100 else x).astype(int)

# Sort dates and return the target dataset
target_df = target_df.sort_values( by=['year','month', 'date'], ascending=True)


In [14]:
'''
* Cross-check the ans.
'''

target_df.head()

,,month,date,year
,match,,,
9,0,4,10,1971
84,0,5,18,1971
2,0,7,8,1971
53,0,7,11,1971
28,0,9,12,1971
